In [187]:
import pandas as pd
import requests
from io import StringIO
import numpy as np

In [188]:
res = requests.get('https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/refs/heads/main/day28-column-transformer/covid_toy.csv').text
df = pd.read_csv(StringIO(res))
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [189]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [190]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

## Long method

In [191]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [192]:
from sklearn.model_selection import train_test_split

In [193]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df.iloc[:, [-1]], test_size=0.2, random_state=42)

In [194]:
x_train_age = x_train.drop(columns=['gender', 'city', 'fever', 'cough']).values
x_test_age = x_test.drop(columns=['gender', 'city', 'fever', 'cough']).values


In [195]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.fit_transform(x_test[['cough']])

In [196]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer()
x_train_fever = imp.fit_transform(x_train[['fever']])
x_test_fever = imp.fit_transform(x_test[['fever']])

In [197]:
from sklearn.preprocessing import OneHotEncoder
oe = OneHotEncoder(drop='first')
x_train_genCity = oe.fit_transform(x_train[['gender', 'city']]).toarray()
x_test_genCity = oe.fit_transform(x_test[['gender', 'city']]).toarray() 

In [198]:
x_train_overall = np.concatenate(([x_train_age, x_train_fever, x_train_genCity, x_train_cough]), axis = 1)
x_test_overall = np.concatenate(([x_test_age, x_test_fever, x_test_genCity, x_test_cough]), axis = 1)
x_train_overall.shape

(80, 7)

## SHORT METHOD

In [199]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers = [
    ('tf1', SimpleImputer(), ['fever']),
    ('tf2', OneHotEncoder(drop='first'), ['gender', 'city']),
    ('tf3', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough'])
], remainder = 'passthrough')

In [200]:
transformer.fit_transform(x_train).shape

(80, 7)